<H1>Overview</H1>

The Passenger Screening Algorithm Challenge asks the data science community to assist with improving threat detection at US airports while minimizing false positives to avoid long lines and delays. (Can I get an amen!).  This notebook is a follow up to my first effort for this contest called [Exploratory Data Analysis and Example Generation](https://www.kaggle.com/jbfarrar/exploratory-data-analysis-and-example-generation) (I'll call it EDA from now on).  As I mentioned in the EDA notebook, the HD-AIT system files supplied in this contest range from 10MB to approximately 2GB per subject.  In the instructions, the organizers suggest that one may even be able to win the contest with one of the smaller image suites. In that notebook, in addition to a review of the data and its vagueries, I supplied some basic building blocks for a preprocessing pipeline.

In this notebook, I continue the series with a full preprocessing pipeline using the building blocks from before as well as a first pass through a CNN based on the Alexnet using Tensorflow.  Clearly, no one is going to win the contest with this method, but I thought it would be helpful to everyone working on this to have an end to end working pipeline.  I hope you find it useful, and if you do, I hope you'll give me an up vote!

As previously noted, I'm not an expert on these systems or the related scans.  If you see something I've misunderstood or you think I've made an error, let me know and I'll correct it.  TSA has made it harder for people to get into this contest by disallowing even masked images to be protrayed on Kaggle, so you'll have to put these scripts in your own environment to take them around the track.  In any event, I am convinced that data science can improve the predictive veracity of these scans.  I'll get off the soap box now and move on.

To begin I collect all of the imports used in the notebook at the top.  It makes it easier when you're converting to a preprocessing script.  Make sure to take note of the last import, tsahelper. You will need to install tsahelper and uncomment this line in order for this pipeline to work. The tsahelper package is made from the EDA and is now available as a pip install (no warranties!). 


In [1]:
# import libraries
from __future__ import print_function
from __future__ import division

import numpy as np 
import pandas as pd
import os
import re
import cv2

import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization

import random
from timeit import default_timer as timer

#import tsahelper as tsa
import tsahelper.tsahelper as tsa

Here I collect the constants all in one place.  Once you are running training routinely, you'll want it to be easy to try different parameters. One tricky thing about the preprocessing approach employed here, is determining when a mini batch is complete.  There are 182 views of the threat zones per subject (more on the terminology and approach for this covered in detail in the [EDA](https://www.kaggle.com/jbfarrar/exploratory-data-analysis-and-example-generation)), so the batch counts depend on this fact to know when we have a complete minibatch.  Note also that FILE_LIST, TRAIN_FILE_LIST, and TEST_FILE_LIST are empty until after preprocessing.  More on that below.

In [2]:
#---------------------------------------------------------------------------------------
# Constants
#
# INPUT_FOLDER:                 The folder that contains the source data
#
# PREPROCESSED_DATA_FOLDER:     The folder that contains preprocessed .npy files 
# 
# STAGE1_LABELS:                The CSV file containing the labels by subject
#
# THREAT_ZONE:                  Threat Zone to train on (actual number not 0 based)
#
# BATCH_SIZE:                   Number of Subjects per batch
#
# EXAMPLES_PER_SUBJECT          Number of examples generated per subject
#
# FILE_LIST:                    A list of the preprocessed .npy files to batch
# 
# TRAIN_TEST_SPLIT_RATIO:       Ratio to split the FILE_LIST between train and test
#
# TRAIN_SET_FILE_LIST:          The list of .npy files to be used for training
#
# TEST_SET_FILE_LIST:           The list of .npy files to be used for testing
#
# IMAGE_DIM:                    The height and width of the images in pixels
#
# LEARNING_RATE                 Learning rate for the neural network
#
# N_TRAIN_STEPS                 The number of train steps (epochs) to run
#
# TRAIN_PATH                    Place to store the tensorboard logs
#
# MODEL_PATH                    Path where model files are stored
#
# MODEL_NAME                    Name of the model files
#
#----------------------------------------------------------------------------------------
INPUT_FOLDER = 'stage1_aps/'
PREPROCESSED_DATA_FOLDER = 'preprocessed/'
PREPROCESSED_NEW_DATA_FOLDER = 'preprocessed_new/'
STAGE1_LABELS = 'stage1_labels.csv'
STAGE1_NEW_LABELS = 'stage1_all.csv'
THREAT_ZONE = 1
BATCH_SIZE = 16
EXAMPLES_PER_SUBJECT = 182

FILE_LIST = []
TRAIN_TEST_SPLIT_RATIO = 0.2
TRAIN_SET_FILE_LIST = []
TEST_SET_FILE_LIST = []

IMAGE_DIM = 250
LEARNING_RATE = 1e-3
N_TRAIN_STEPS = 1
TRAIN_PATH = 'tsalogs/train/'
MODEL_PATH = 'tsalogs/model/'
MODEL_NAME = ('tsa-{}-lr-{}-{}-{}-tz-{}'.format('alexnet-v0.1', LEARNING_RATE, IMAGE_DIM, 
                                                IMAGE_DIM, THREAT_ZONE )) 


<H3>The Preprocessor</H3>

Throughout this notebook, passengers who are being scanned for contraband are referred to as "subjects".  The preprocessor begins with 3 different ways you can choose to read in a list of subjects.  If you are training on the full data set, OPTION 1 (see the comments) is your best bet.  If you want to preprocess all subjects for whom you have data, then OPTION 2 is your best choice, and if you are running in a sample or low volume notebook environment, then you can just give a short list of subject IDs for whom you have data loaded using OPTION 3.

My approach in this notebook is to isolate each individual threat zone from every visible angle and then make features out of each individual threat zone from each angle that a given threat zone is visible. This allows us to train on each threat zone individually from every view in a 2D format.  (This is covered in some detail in the [EDA](https://www.kaggle.com/jbfarrar/exploratory-data-analysis-and-example-generation)).

The preprocessor loops through the data one subject at a time, transforms the images, isolates threat zones, and uses a set of vertices to crop each image to 250x250.  Images are saved in minibatches by threat zone, so that they can be read into the trainer.

Note that the trainer depends upon the threat zone number being present in the minibatch file name.  (Not my favorite approach, but it was fastest and easiest given what I was doing).  If you have a better idea, pass it along!

In [3]:
def threshold(a, threshmin=None, threshmax=None, newval=0):
    a = np.ma.array(a, copy=True)
    mask = np.zeros(a.shape, dtype=bool)
    if threshmin is not None:
        mask |= (a < threshmin).filled(False)

    if threshmax is not None:
        mask |= (a > threshmax).filled(False)

    a[mask] = newval
    return a
#-------------------------------------------------------------------------------
# spread_spectrum(img):        applies a histogram equalization transformation
#
# img:                         a single scan
#
# returns:                     a transformed scan
#-------------------------------------------------------------------------------

def spread_spectrum(img):
    #img = stats.threshold(img, threshmin=12, newval=0)
    img = threshold(img, threshmin=12, newval=0)
    
    # see http://docs.opencv.org/3.1.0/d5/daf/tutorial_py_histogram_equalization.html
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img= clahe.apply(img)
    
    return img
  
# unit test ------------------------------------------
#img_high_contrast = spread_spectrum(img_rescaled)

#fig, axarr = plt.subplots(nrows=1, ncols=2, figsize=(20, 5))

#axarr[0].imshow(img_high_contrast, cmap=COLORMAP)
#plt.subplot(122)
#plt.hist(img_high_contrast.flatten(), bins=256, color='c')
#plt.xlabel("Grayscale Pixel Value")
#plt.ylabel("Frequency")
#plt.show()


In [4]:
input_v = 'stage1_sample_submission.csv'
sub_list = pd.read_csv(input_v)
sub_list['Subject'], sub_list['Zone'] = sub_list['Id'].str.split('_',1).str
sub_list = sub_list['Subject'].unique()
sub_list = sub_list.tolist()
sub_list[0:20]

['0367394485447c1c3485359ba71f52cb',
 '0749d03b6d2d1eebb6c24ecb3c36e645',
 '08dc52d08482393070d8c71b11902636',
 '0a83698bce92a6824dcc37c1d7fc31f5',
 '0b2996d90c7250a3acb63cab6ae5bebe',
 '0fadb2ac7be1a0ed6fb7cd81372c18ba',
 '0fd54393dce1f55b5a4d7e30b8642158',
 '104e284349ffe68378745fc8c5638eab',
 '123d65c19a4d7f395b3719d47d5666b5',
 '1240199d7b5d6d1c709b13381a198b75',
 '12ed0db15024397106482c4983ad2d92',
 '1b861c23fc370c326ec8342733ef5d84',
 '1cb13f156bd436222447dd658180bd96',
 '1ccf082d6f3d51bd55113fb092be0e59',
 '1cef83028272a04070fbdffae5796a87',
 '249092cc1775fabcb79ba439ba974759',
 '27069e6977eb7e0ccd913af254932e39',
 '28970155f9954bbd7d5c737b60a66b5a',
 '2b81524a1a36b6b91baf7c84dc16b29b',
 '334cfc11aceab7e8bbd1d711f70c514d']

In [5]:
#---------------------------------------------------------------------------------------
# preprocess_tsa_data(): preprocesses the tsa datasets
#
# parameters:      none
#
# returns:         none
#---------------------------------------------------------------------------------------

def preprocess_tsa_data():
    
    # OPTION 1: get a list of all subjects for which there are labels
    #df = pd.read_csv(STAGE1_LABELS)
    #df['Subject'], df['Zone'] = df['Id'].str.split('_',1).str
    #SUBJECT_LIST = df['Subject'].unique()

    # OPTION 2: get a list of all subjects for whom there is data
    #SUBJECT_LIST = [os.path.splitext(subject)[0] for subject in os.listdir(INPUT_FOLDER)]
    
    # OPTION 3: get a list of subjects for small bore test purposes
    SUBJECT_LIST = ['00360f79fd6e02781457eda48f85da90','0043db5e8c819bffc15261b1f1ac5e42',
                    '0050492f92e22eed3474ae3a6fc907fa','006ec59fa59dd80a64c85347eef810c7',
                    '0097503ee9fa0606559c56458b281a08','011516ab0eca7cad7f5257672ddde70e']
    
    # intialize tracking and saving items
    batch_num = 1
    threat_zone_examples = []
    start_time = timer()
    
    for subject in SUBJECT_LIST:

        # read in the images
        print('--------------------------------------------------------------')
        print('t+> {:5.3f} |Reading images for subject #: {}'.format(timer()-start_time, 
                                                                     subject))
        print('--------------------------------------------------------------')
        images = tsa.read_data(INPUT_FOLDER + '/' + subject + '.aps')

        # transpose so that the slice is the first dimension shape(16, 620, 512)
        images = images.transpose()

        # for each threat zone, loop through each image, mask off the zone and then crop it
        for tz_num, threat_zone_x_crop_dims in enumerate(zip(tsa.zone_slice_list, 
                                                             tsa.zone_crop_list)):

            threat_zone = threat_zone_x_crop_dims[0]
            crop_dims = threat_zone_x_crop_dims[1]

            # get label
            label = np.array(tsa.get_subject_zone_label(tz_num, 
                             tsa.get_subject_labels(STAGE1_LABELS, subject)))

            for img_num, img in enumerate(images):

                print('Threat Zone:Image -> {}:{}'.format(tz_num, img_num))
                print('Threat Zone Label -> {}'.format(label))
                
                if threat_zone[img_num] is not None:

                    # correct the orientation of the image
                    print('-> reorienting base image') 
                    base_img = np.flipud(img)
                    print('-> shape {}|mean={}'.format(base_img.shape, 
                                                       base_img.mean()))

                    # convert to grayscale
                    print('-> converting to grayscale')
                    rescaled_img = tsa.convert_to_grayscale(base_img)
                    print('-> shape {}|mean={}'.format(rescaled_img.shape, 
                                                       rescaled_img.mean()))

                    # spread the spectrum to improve contrast
                    print('-> spreading spectrum')
                    #high_contrast_img = tsa.spread_spectrum(rescaled_img)
                    high_contrast_img = spread_spectrum(rescaled_img)
                    print('-> shape {}|mean={}'.format(high_contrast_img.shape,
                                                       high_contrast_img.mean()))

                    # get the masked image
                    print('-> masking image')
                    masked_img = tsa.roi(high_contrast_img, threat_zone[img_num])
                    print('-> shape {}|mean={}'.format(masked_img.shape, 
                                                       masked_img.mean()))

                    # crop the image
                    print('-> cropping image')
                    cropped_img = tsa.crop(masked_img, crop_dims[img_num])
                    print('-> shape {}|mean={}'.format(cropped_img.shape, 
                                                       cropped_img.mean()))

                    # normalize the image
                    print('-> normalizing image')
                    normalized_img = tsa.normalize(cropped_img)
                    print('-> shape {}|mean={}'.format(normalized_img.shape, 
                                                       normalized_img.mean()))

                    # zero center the image
                    print('-> zero centering')
                    zero_centered_img = tsa.zero_center(normalized_img)
                    print('-> shape {}|mean={}'.format(zero_centered_img.shape, 
                                                       zero_centered_img.mean()))

                    # append the features and labels to this threat zone's example array
                    print ('-> appending example to threat zone {}'.format(tz_num))
                    threat_zone_examples.append([[tz_num], zero_centered_img, label])
                    print ('-> shape {:d}:{:d}:{:d}:{:d}:{:d}:{:d}'.format(
                                                         len(threat_zone_examples),
                                                         len(threat_zone_examples[0]),
                                                         len(threat_zone_examples[0][0]),
                                                         len(threat_zone_examples[0][1][0]),
                                                         len(threat_zone_examples[0][1][1]),
                                                         len(threat_zone_examples[0][2])))
                else:
                    print('-> No view of tz:{} in img:{}. Skipping to next...'.format( 
                                tz_num, img_num))
                print('------------------------------------------------')

        # each subject gets EXAMPLES_PER_SUBJECT number of examples (182 to be exact, 
        # so this section just writes out the the data once there is a full minibatch 
        # complete.
        if ((len(threat_zone_examples) % (BATCH_SIZE * EXAMPLES_PER_SUBJECT)) == 0):
            for tz_num, tz in enumerate(tsa.zone_slice_list):

                tz_examples_to_save = []

                # write out the batch and reset
                print(' -> writing: ' + PREPROCESSED_DATA_FOLDER + 
                                        'preprocessed_TSA_scans-tz{}-{}-{}-b{}.npy'.format( 
                                        tz_num+1,
                                        len(threat_zone_examples[0][1][0]),
                                        len(threat_zone_examples[0][1][1]), 
                                        batch_num))

                # get this tz's examples
                tz_examples = [example for example in threat_zone_examples if example[0] == [tz_num]]

                # drop unused columns
                tz_examples_to_save.append([[features_label[1], features_label[2]] 
                                            for features_label in tz_examples])

                # save batch.  Note that the trainer looks for tz{} where {} is a 
                # tz_num 1 based in the minibatch file to select which batches to 
                # use for training a given threat zone
                np.save(PREPROCESSED_DATA_FOLDER +  'preprocessed_TSA_scans-tz{}-{}-{}-b{}.npy'.format(tz_num+1, len(threat_zone_examples[0][1][0]),len(threat_zone_examples[0][1][1]),batch_num), tz_examples_to_save)
                del tz_examples_to_save

            #reset for next batch 
            del threat_zone_examples
            threat_zone_examples = []
            batch_num += 1
    
    # we may run out of subjects before we finish a batch, so we write out 
    # the last batch stub
    if (len(threat_zone_examples) > 0):
        for tz_num, tz in enumerate(tsa.zone_slice_list):

            tz_examples_to_save = []

            # write out the batch and reset
            print(' -> writing: ' + PREPROCESSED_DATA_FOLDER 
                    + 'preprocessed_TSA_scans-tz{}-{}-{}-b{}.npy'.format(tz_num+1, 
                      len(threat_zone_examples[0][1][0]),
                      len(threat_zone_examples[0][1][1]), 
                                                                                                                  batch_num))

            # get this tz's examples
            tz_examples = [example for example in threat_zone_examples if example[0] == 
                           [tz_num]]

            # drop unused columns
            tz_examples_to_save.append([[features_label[1], features_label[2]] 
                                        for features_label in tz_examples])

            #save batch
            np.save(PREPROCESSED_DATA_FOLDER + 
                    'preprocessed_TSA_scans-tz{}-{}-{}-b{}.npy'.format(tz_num+1, 
                                                     len(threat_zone_examples[0][1][0]),
                                                     len(threat_zone_examples[0][1][1]), 
                                                     batch_num), 
                                                     tz_examples_to_save)
# unit test ---------------------------------------
#preprocess_tsa_data()

In [6]:
preprocess_tsa_data()

--------------------------------------------------------------
t+> 0.000 |Reading images for subject #: 00360f79fd6e02781457eda48f85da90
--------------------------------------------------------------
Threat Zone:Image -> 0:0
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=5.595098627964035e-05
-> converting to grayscale
-> shape (660, 512)|mean=12.65384706439394
-> spreading spectrum
-> shape (660, 512)|mean=18.996087831439393
-> masking image
-> shape (660, 512)|mean=0.7072354403409091
-> cropping image
-> shape (250, 250)|mean=3.653424
-> normalizing image
-> shape (250, 250)|mean=0.014327152941176471
-> zero centering
-> shape (250, 250)|mean=1.5294117646749328e-07
-> appending example to threat zone 0
-> shape 1:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 0:1
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=5.5450127547373995e-05
-> converting to grayscale
-> shape (660, 512)|mean=14.

-> shape (660, 512)|mean=13.771875
-> spreading spectrum
-> shape (660, 512)|mean=19.722727272727273
-> masking image
-> shape (660, 512)|mean=0.5068684895833333
-> cropping image
-> shape (250, 250)|mean=2.740496
-> normalizing image
-> shape (250, 250)|mean=0.010747043137254903
-> zero centering
-> shape (250, 250)|mean=-0.003579956862745099
-> appending example to threat zone 1
-> shape 14:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 1:3
Threat Zone Label -> [1 0]
-> No view of tz:1 in img:3. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 1:4
Threat Zone Label -> [1 0]
-> No view of tz:1 in img:4. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 1:5
Threat Zone Label -> [1 0]
-> No view of tz:1 in img:5. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 1:6
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660

-> shape (660, 512)|mean=15.024221709280303
-> spreading spectrum
-> shape (660, 512)|mean=22.83842921401515
-> masking image
-> shape (660, 512)|mean=0.5337357954545454
-> cropping image
-> shape (250, 250)|mean=2.71536
-> normalizing image
-> shape (250, 250)|mean=0.010648470588235294
-> zero centering
-> shape (250, 250)|mean=-0.003678529411764709
-> appending example to threat zone 2
-> shape 30:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 2:10
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=6.145900988485664e-05
-> converting to grayscale
-> shape (660, 512)|mean=15.221875
-> spreading spectrum
-> shape (660, 512)|mean=21.414867424242424
-> masking image
-> shape (660, 512)|mean=0.2697147253787879
-> cropping image
-> shape (250, 250)|mean=1.287872
-> normalizing image
-> shape (250, 250)|mean=0.0050504784313725485
-> zero centering
-> shape (250, 250)|mean=-0.009276521568627454
-> appending example to threat zon

-> normalizing image
-> shape (250, 250)|mean=0.00781364705882353
-> zero centering
-> shape (250, 250)|mean=-0.006513352941176471
-> appending example to threat zone 3
-> shape 44:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 3:12
Threat Zone Label -> [1 0]
-> No view of tz:3 in img:12. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 3:13
Threat Zone Label -> [1 0]
-> No view of tz:3 in img:13. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 3:14
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=5.303452053340152e-05
-> converting to grayscale
-> shape (660, 512)|mean=13.652367424242424
-> spreading spectrum
-> shape (660, 512)|mean=19.183093631628786
-> masking image
-> shape (660, 512)|mean=0.23387784090909092
-> cropping image
-> shape (250, 250)|mean=0.0
-> normalizing image
-> shape (250, 250)|mean=0.0
-> zero centering
-> shape (25

-> shape (660, 512)|mean=12.958445785984848
-> spreading spectrum
-> shape (660, 512)|mean=18.86434659090909
-> masking image
-> shape (660, 512)|mean=1.5361120975378788
-> cropping image
-> shape (250, 250)|mean=8.012528
-> normalizing image
-> shape (250, 250)|mean=0.03142167843137255
-> zero centering
-> shape (250, 250)|mean=0.017094678431372544
-> appending example to threat zone 5
-> shape 61:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 5:14
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=5.303452053340152e-05
-> converting to grayscale
-> shape (660, 512)|mean=13.652367424242424
-> spreading spectrum
-> shape (660, 512)|mean=19.183093631628786
-> masking image
-> shape (660, 512)|mean=1.7978308475378788
-> cropping image
-> shape (250, 250)|mean=9.427568
-> normalizing image
-> shape (250, 250)|mean=0.03697085490196078
-> zero centering
-> shape (250, 250)|mean=0.022643854901960782
-> appending example to threa

...

-> shape (660, 512)|mean=10.295069839015152
-> spreading spectrum
-> shape (660, 512)|mean=15.528568892045454
-> masking image
-> shape (660, 512)|mean=2.113278290719697
-> cropping image
-> shape (250, 250)|mean=11.199728
-> normalizing image
-> shape (250, 250)|mean=0.043920501960784314
-> zero centering
-> shape (250, 250)|mean=0.029593501960784314
-> appending example to threat zone 11
-> shape 1026:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 11:4
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=3.618556002038531e-05
-> converting to grayscale
-> shape (660, 512)|mean=8.516574928977272
-> spreading spectrum
-> shape (660, 512)|mean=12.763334517045454
-> masking image
-> shape (660, 512)|mean=1.5536517518939394
-> cropping image
-> shape (250, 250)|mean=8.173984
-> normalizing image
-> shape (250, 250)|mean=0.03205483921568627
-> zero centering
-> shape (250, 250)|mean=0.017727839215686277
-> appending example to t

-> shape (660, 512)|mean=1.01484375
-> cropping image
-> shape (250, 250)|mean=5.2608
-> normalizing image
-> shape (250, 250)|mean=0.020630588235294116
-> zero centering
-> shape (250, 250)|mean=0.006303588235294116
-> appending example to threat zone 12
-> shape 1042:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 12:8
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=5.961389615549706e-05
-> converting to grayscale
-> shape (660, 512)|mean=15.597706557765152
-> spreading spectrum
-> shape (660, 512)|mean=21.090512547348485
-> masking image
-> shape (660, 512)|mean=0.9598751183712121
-> cropping image
-> shape (250, 250)|mean=4.9636
-> normalizing image
-> shape (250, 250)|mean=0.019465098039215686
-> zero centering
-> shape (250, 250)|mean=0.005138098039215684
-> appending example to threat zone 12
-> shape 1043:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 12:9
Threat Zone Label ->

-> shape (660, 512)|mean=17.790263967803032
-> masking image
-> shape (660, 512)|mean=1.3769146543560606
-> cropping image
-> shape (250, 250)|mean=6.555296
-> normalizing image
-> shape (250, 250)|mean=0.0257070431372549
-> zero centering
-> shape (250, 250)|mean=0.011380043137254896
-> appending example to threat zone 13
-> shape 1055:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 13:7
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=5.426221832749434e-05
-> converting to grayscale
-> shape (660, 512)|mean=16.98489287405303
-> spreading spectrum
-> shape (660, 512)|mean=21.45642459753788
-> masking image
-> shape (660, 512)|mean=1.2949455492424242
-> cropping image
-> shape (250, 250)|mean=3.190624
-> normalizing image
-> shape (250, 250)|mean=0.012512250980392157
-> zero centering
-> shape (250, 250)|mean=-0.0018147490196078464
-> appending example to threat zone 13
-> shape 1056:3:1:250:250:2
------------------------

-> shape (660, 512)|mean=12.23960996685606
-> spreading spectrum
-> shape (660, 512)|mean=16.988420336174244
-> masking image
-> shape (660, 512)|mean=0.44289772727272725
-> cropping image
-> shape (250, 250)|mean=2.365232
-> normalizing image
-> shape (250, 250)|mean=0.009275419607843135
-> zero centering
-> shape (250, 250)|mean=-0.005051580392156866
-> appending example to threat zone 14
-> shape 1070:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 14:15
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=4.65764787804801e-05
-> converting to grayscale
-> shape (660, 512)|mean=12.684096827651516
-> spreading spectrum
-> shape (660, 512)|mean=17.68485144412879
-> masking image
-> shape (660, 512)|mean=0.3638198390151515
-> cropping image
-> shape (250, 250)|mean=1.945984
-> normalizing image
-> shape (250, 250)|mean=0.0076313098039215675
-> zero centering
-> shape (250, 250)|mean=-0.0066956901960784345
-> appending example

Threat Zone:Image -> 16:0
Threat Zone Label -> [1 0]
-> No view of tz:16 in img:0. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 16:1
Threat Zone Label -> [1 0]
-> No view of tz:16 in img:1. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 16:2
Threat Zone Label -> [1 0]
-> No view of tz:16 in img:2. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 16:3
Threat Zone Label -> [1 0]
-> No view of tz:16 in img:3. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 16:4
Threat Zone Label -> [1 0]
-> No view of tz:16 in img:4. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 16:5
Threat Zone Label -> [1 0]
-> No view of tz:16 in img:5. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 16:6
Threat Zone Label -> [1 0]
-> No view of tz:16 in img:6. Skipp

<H3>Train and Test Split</H3>

The next function takes the full minibatch list and splits it between train and test sets, using the TRAIN_TEST_SPLIT_RATIO.  Note that as mentioned above building FILE_LIST searches through the minibatch file name and looks for the string '-tz' + THREAT_ZONE + '-' in the file name.  If you used the preprocessor above, it creates the files in that form.

In [25]:
#---------------------------------------------------------------------------------------
# get_train_test_file_list(): gets the batch file list, splits between train and test
#
# parameters:      none
#
# returns:         none
#
#-------------------------------------------------------------------------------------

def get_train_test_file_list():
    
    global FILE_LIST
    global TRAIN_SET_FILE_LIST
    global TEST_SET_FILE_LIST

    if os.listdir(PREPROCESSED_DATA_FOLDER) == []:
        print ('No preprocessed data available.  Skipping preprocessed data setup..')
    else:
        #FILE_LIST = [f for f in os.listdir(PREPROCESSED_DATA_FOLDER) 
        #             if re.search(re.compile('-tz' + str(THREAT_ZONE) + '-'), f)]
        FILE_LIST = [f for f in os.listdir(PREPROCESSED_DATA_FOLDER)]
                    # if re.search(re.compile('-tz' + str(THREAT_ZONE) + '-'), f)]
        train_test_split = len(FILE_LIST) - \
                           max(int(len(FILE_LIST)*TRAIN_TEST_SPLIT_RATIO),1)
        TRAIN_SET_FILE_LIST = FILE_LIST[:train_test_split]
        TEST_SET_FILE_LIST = FILE_LIST[train_test_split:]
        TRAIN_SET_FILE_LIST = TEST_SET_FILE_LIST
        print('Train/Test Split -> {} file(s) of {} used for testing'.format( 
              len(FILE_LIST) - train_test_split, len(FILE_LIST)))
        
# unit test ----------------------------
#get_train_test_file_list()
#print (

In [26]:
get_train_test_file_list()

Train/Test Split -> 3 file(s) of 17 used for testing


In [27]:
TRAIN_SET_FILE_LIST

['preprocessed_TSA_scans-tz7-250-250-b1.npy',
 'preprocessed_TSA_scans-tz8-250-250-b1.npy',
 'preprocessed_TSA_scans-tz9-250-250-b1.npy']

<H3>Generating an Input Pipeline</H3>

The following function reads in a minibatch, extracts features and labels, and then returns the data in a form that can be easily streamed into a tensorfow feed dictionary, or as we will do below, as a feed dictionary to a TFLearn based CNN.

In [11]:
#---------------------------------------------------------------------------------------
# input_pipeline(filename, path): prepares a batch of features and labels for training
#
# parameters:      filename - the file to be batched into the model
#                  path - the folder where filename resides
#
# returns:         feature_batch - a batch of features to train or test on
#                  label_batch - a batch of labels related to the feature_batch
#
#---------------------------------------------------------------------------------------

def input_pipeline(filename, path):

    preprocessed_tz_scans = []
    feature_batch = []
    label_batch = []
    
    #Load a batch of preprocessed tz scans
    preprocessed_tz_scans = np.load(os.path.join(path, filename))
        
    #Shuffle to randomize for input into the model
    np.random.shuffle(preprocessed_tz_scans)
    
    # separate features and labels
    for example_list in preprocessed_tz_scans:
        for example in example_list:
            feature_batch.append(example[0])
            label_batch.append(example[1])
    
    feature_batch = np.asarray(feature_batch, dtype=np.float32)
    label_batch = np.asarray(label_batch, dtype=np.float32)
    
    return feature_batch, label_batch
  
# unit test ------------------------------------------------------------------------
#print ('Train Set -----------------------------')
#for f_in in TRAIN_SET_FILE_LIST:
#    feature_batch, label_batch = input_pipeline(f_in, PREPROCESSED_DATA_FOLDER)
#    print (' -> features shape {}:{}:{}'.format(len(feature_batch), 
#                                                len(feature_batch[0]), 
#                                                len(feature_batch[0][0])))
#    print (' -> labels shape   {}:{}'.format(len(label_batch), len(label_batch[0])))
    
#print ('Test Set -----------------------------')
#for f_in in TEST_SET_FILE_LIST:
#    feature_batch, label_batch = input_pipeline(f_in, PREPROCESSED_DATA_FOLDER)
#    print (' -> features shape {}:{}:{}'.format(len(feature_batch), 
#                                                len(feature_batch[0]), 
#                                                len(feature_batch[0][0])))
#    print (' -> labels shape   {}:{}'.format(len(label_batch), len(label_batch[0])))

In [12]:
print ('Train Set -----------------------------')
for f_in in TRAIN_SET_FILE_LIST:
    feature_batch, label_batch = input_pipeline(f_in, PREPROCESSED_DATA_FOLDER)
    print (' -> features shape {}:{}:{}'.format(len(feature_batch), len(feature_batch[0]),len(feature_batch[0][0])))
    print (' -> labels shape   {}:{}'.format(len(label_batch), len(label_batch[0])))
    
print ('Test Set -----------------------------')
for f_in in TEST_SET_FILE_LIST:
    feature_batch, label_batch = input_pipeline(f_in, PREPROCESSED_DATA_FOLDER)
    print (' -> features shape {}:{}:{}'.format(len(feature_batch), len(feature_batch[0]), len(feature_batch[0][0])))
    print (' -> labels shape   {}:{}'.format(len(label_batch), len(label_batch[0])))

Train Set -----------------------------
 -> features shape 48:250:250
 -> labels shape   48:2
 -> features shape 66:250:250
 -> labels shape   66:2
 -> features shape 54:250:250
 -> labels shape   54:2
Test Set -----------------------------
 -> features shape 48:250:250
 -> labels shape   48:2
 -> features shape 66:250:250
 -> labels shape   66:2
 -> features shape 54:250:250
 -> labels shape   54:2


<H3>Shuffling the Training Set</H3>

Below we use TFLearn, an abstraction of Tensorflow, to build the convnet. Using TFLearn we can set the fit operation to shuffle rows within a mini batch.  This function shuffles the minibatch list, so that in addition to intra-minibatch shuffling, the there is also  shuffling of the order the mini batches are fed to the model.

In [13]:
#---------------------------------------------------------------------------------------
# shuffle_train_set(): shuffle the list of batch files so that each train step
#                      receives them in a different order since the TRAIN_SET_FILE_LIST
#                      is a global
#
# parameters:      train_set - the file listing to be shuffled
#
# returns:         none
#
#-------------------------------------------------------------------------------------

def shuffle_train_set(train_set):
    sorted_file_list = random.shuffle(train_set)
    TRAIN_SET_FILE_LIST = sorted_file_list
    
# Unit test ---------------
#print ('Before Shuffling ->', TRAIN_SET_FILE_LIST)
#shuffle_train_set(TRAIN_SET_FILE_LIST)
#print ('After Shuffling ->', TRAIN_SET_FILE_LIST)

In [14]:
print ('Before Shuffling ->', TRAIN_SET_FILE_LIST)
shuffle_train_set(TRAIN_SET_FILE_LIST)
print ('After Shuffling ->', TRAIN_SET_FILE_LIST)

Before Shuffling -> ['preprocessed_TSA_scans-tz7-250-250-b1.npy', 'preprocessed_TSA_scans-tz8-250-250-b1.npy', 'preprocessed_TSA_scans-tz9-250-250-b1.npy']
After Shuffling -> ['preprocessed_TSA_scans-tz8-250-250-b1.npy', 'preprocessed_TSA_scans-tz9-250-250-b1.npy', 'preprocessed_TSA_scans-tz7-250-250-b1.npy']


<H3>Defining the Alexnet CNN</H3>

The Alexnet was first put to the real world test during the ImageNet Large Scale Visual Recognition Challenge in 2012. The performance of this network was a quantum shift for its time as the model achieved a top-5 error of 15.3%, more than 10.8 percentage points ahead of the runner up.  The solution is elaborated in  [this paper by the original author](https://www.nvidia.cn/content/tesla/pdf/machine-learning/imagenet-classification-with-deep-convolutional-nn.pdf) if you are interested in learning more. 

But in short the network consists of 7 layers, 5 convolutions/maxpools, plus 2 regression layers at the end.  The structure of the model looks like this: 

<img src="https://kratzert.github.io/images/finetune_alexnet/alexnet.png" width="700" height="600">

Using TFLearn makes this definition is quite intuitive and simple.  

In [15]:
#---------------------------------------------------------------------------------------
# alexnet(width, height, lr): defines the alexnet
#
# parameters:      width - width of the input image
#                  height - height of the input image
#                  lr - learning rate
#
# returns:         none
#
#-------------------------------------------------------------------------------------

def alexnet(width, height, lr):
    network = input_data(shape=[None, width, height, 1], name='features')
    network = conv_2d(network, 96, 11, strides=4, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='momentum', loss='categorical_crossentropy', 
                         learning_rate=lr, name='labels')

    model = tflearn.DNN(network, checkpoint_path=MODEL_PATH + MODEL_NAME, 
                        tensorboard_dir=TRAIN_PATH, tensorboard_verbose=3, max_checkpoints=1)

    return model

<H3>The Trainer</H3>

Finally, the trainer is straight forward.  Set up the network, loop to read in minibatches for test and train and run the fit method.  Note that TFLearn treats each "minibatch" as an epoch.  For the illustration purposes noted here, its not a big deal, but after may runs it may be quite annoying.  

Up until now, all the work I've personally done has been using the lower level interface.  This was my first time trying TFLearn.  I liked a lot about TFLearn.  Network construction is easy-peasy.  I haven't worked with Keras as of yet, but it looks like it may have a few advantages worth considering.

In [16]:
#---------------------------------------------------------------------------------------
# train_conv_net(): runs the train op
#
# parameters:      none
#
# returns:         none
#
#-------------------------------------------------------------------------------------

#def train_conv_net():
    
val_features = []
val_labels = []

# get train and test batches
get_train_test_file_list()

# instantiate model
model = alexnet(IMAGE_DIM, IMAGE_DIM, LEARNING_RATE)

# read in the validation test set
for j, test_f_in in enumerate(TEST_SET_FILE_LIST):
    if j == 0:
        val_features, val_labels = input_pipeline(test_f_in, PREPROCESSED_DATA_FOLDER)
    else:
        tmp_feature_batch, tmp_label_batch = input_pipeline(test_f_in, 
                                                            PREPROCESSED_DATA_FOLDER)
        val_features = np.concatenate((tmp_feature_batch, val_features), axis=0)
        val_labels = np.concatenate((tmp_label_batch, val_labels), axis=0)

val_features = val_features.reshape(-1, IMAGE_DIM, IMAGE_DIM, 1)

# start training process
for i in range(N_TRAIN_STEPS):

    # shuffle the train set files before each step
    shuffle_train_set(TRAIN_SET_FILE_LIST)

    # run through every batch in the training set
    for f_in in TRAIN_SET_FILE_LIST:

        # read in a batch of features and labels for training
        feature_batch, label_batch = input_pipeline(f_in, PREPROCESSED_DATA_FOLDER)
        feature_batch = feature_batch.reshape(-1, IMAGE_DIM, IMAGE_DIM, 1)
        print ('Feature Batch Shape ->', feature_batch.shape)                

        # run the fit operation
        print(model.fit({'features': feature_batch}, {'labels': label_batch}, n_epoch=1, 
                  validation_set=({'features': val_features}, {'labels': val_labels}), 
                  shuffle=True, snapshot_step=None, show_metric=True, 
                  run_id=MODEL_NAME))

        
# unit test -----------------------------------
#train_conv_net()

Training Step: 1  | time: 35.838s
| Momentum | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 64/66
Training Step: 2  | total loss: 0.53876 | time: 58.256s
| Momentum | epoch: 001 | loss: 0.53876 - acc: 0.6187 | val_loss: 0.49158 - val_acc: 0.8810 -- iter: 66/66
--
None
Feature Batch Shape -> (54, 250, 250, 1)
---------------------------------
Run id: tsa-alexnet-v0.1-lr-0.001-250-250-tz-1
Log directory: tsalogs/train/
---------------------------------
Training samples: 54
Validation samples: 168
--
Training Step: 3  | total loss: 0.54352 | time: 47.007s
| Momentum | epoch: 002 | loss: 0.54352 - acc: 0.9307 | val_loss: 0.44377 - val_acc: 0.8810 -- iter: 54/54
--
None
Feature Batch Shape -> (48, 250, 250, 1)
---------------------------------
Run id: tsa-alexnet-v0.1-lr-0.001-250-250-tz-1
Log directory: tsalogs/train/
---------------------------------
Training samples: 48
Validation samples: 168
--
Training Step: 4  | total loss: 0.55167 | time: 41.038s
| Momentum | epoch: 003 | loss:

In [128]:
#---------------------------------------------------------------------------------------
# preprocess_tsa_data(): preprocesses the tsa datasets
#
# parameters:      none
#
# returns:         none
#---------------------------------------------------------------------------------------

def preprocess_tsa_new_data():
    
    # OPTION 1: get a list of all subjects for which there are labels
    #df = pd.read_csv(STAGE1_LABELS)
    #df['Subject'], df['Zone'] = df['Id'].str.split('_',1).str
    #SUBJECT_LIST = df['Subject'].unique()

    # OPTION 2: get a list of all subjects for whom there is data
    #SUBJECT_LIST = [os.path.splitext(subject)[0] for subject in os.listdir(INPUT_FOLDER)]
    
    # OPTION 3: get a list of subjects for small bore test purposes
    #SUBJECT_LIST = ['00360f79fd6e02781457eda48f85da90','0043db5e8c819bffc15261b1f1ac5e42',
    #                '0050492f92e22eed3474ae3a6fc907fa','006ec59fa59dd80a64c85347eef810c7',
    #                '0097503ee9fa0606559c56458b281a08','011516ab0eca7cad7f5257672ddde70e']
    
    SUBJECT_LIST = sub_list[0:1]
    
    # intialize tracking and saving items
    batch_num = 1
    threat_zone_examples = []
    start_time = timer()
    
    for subject in SUBJECT_LIST:

        # read in the images
        print('--------------------------------------------------------------')
        print('t+> {:5.3f} |Reading images for subject #: {}'.format(timer()-start_time, 
                                                                     subject))
        print('--------------------------------------------------------------')
        images = tsa.read_data(INPUT_FOLDER + '/' + subject + '.aps')

        # transpose so that the slice is the first dimension shape(16, 620, 512)
        images = images.transpose()

        # for each threat zone, loop through each image, mask off the zone and then crop it
        for tz_num, threat_zone_x_crop_dims in enumerate(zip(tsa.zone_slice_list, 
                                                             tsa.zone_crop_list)):

            threat_zone = threat_zone_x_crop_dims[0]
            crop_dims = threat_zone_x_crop_dims[1]

            # get label
            label = np.array(tsa.get_subject_zone_label(tz_num, 
                             tsa.get_subject_labels(STAGE1_NEW_LABELS, subject)))

            for img_num, img in enumerate(images):

                print('Threat Zone:Image -> {}:{}'.format(tz_num, img_num))
                print('Threat Zone Label -> {}'.format(label))
                
                if threat_zone[img_num] is not None:

                    # correct the orientation of the image
                    print('-> reorienting base image') 
                    base_img = np.flipud(img)
                    print('-> shape {}|mean={}'.format(base_img.shape, 
                                                       base_img.mean()))

                    # convert to grayscale
                    print('-> converting to grayscale')
                    rescaled_img = tsa.convert_to_grayscale(base_img)
                    print('-> shape {}|mean={}'.format(rescaled_img.shape, 
                                                       rescaled_img.mean()))

                    # spread the spectrum to improve contrast
                    print('-> spreading spectrum')
                    #high_contrast_img = tsa.spread_spectrum(rescaled_img)
                    high_contrast_img = spread_spectrum(rescaled_img)
                    print('-> shape {}|mean={}'.format(high_contrast_img.shape,
                                                       high_contrast_img.mean()))

                    # get the masked image
                    print('-> masking image')
                    masked_img = tsa.roi(high_contrast_img, threat_zone[img_num])
                    print('-> shape {}|mean={}'.format(masked_img.shape, 
                                                       masked_img.mean()))

                    # crop the image
                    print('-> cropping image')
                    cropped_img = tsa.crop(masked_img, crop_dims[img_num])
                    print('-> shape {}|mean={}'.format(cropped_img.shape, 
                                                       cropped_img.mean()))

                    # normalize the image
                    print('-> normalizing image')
                    normalized_img = tsa.normalize(cropped_img)
                    print('-> shape {}|mean={}'.format(normalized_img.shape, 
                                                       normalized_img.mean()))

                    # zero center the image
                    print('-> zero centering')
                    zero_centered_img = tsa.zero_center(normalized_img)
                    print('-> shape {}|mean={}'.format(zero_centered_img.shape, 
                                                       zero_centered_img.mean()))

                    # append the features and labels to this threat zone's example array
                    print ('-> appending example to threat zone {}'.format(tz_num))
                    threat_zone_examples.append([[tz_num], zero_centered_img, label])
                    print ('-> shape {:d}:{:d}:{:d}:{:d}:{:d}:{:d}'.format(
                                                         len(threat_zone_examples),
                                                         len(threat_zone_examples[0]),
                                                         len(threat_zone_examples[0][0]),
                                                         len(threat_zone_examples[0][1][0]),
                                                         len(threat_zone_examples[0][1][1]),
                                                         len(threat_zone_examples[0][2])))
                else:
                    print('-> No view of tz:{} in img:{}. Skipping to next...'.format( 
                                tz_num, img_num))
                print('------------------------------------------------')

        # each subject gets EXAMPLES_PER_SUBJECT number of examples (182 to be exact, 
        # so this section just writes out the the data once there is a full minibatch 
        # complete.
        if ((len(threat_zone_examples) % (BATCH_SIZE * EXAMPLES_PER_SUBJECT)) == 0):
            for tz_num, tz in enumerate(tsa.zone_slice_list):

                tz_examples_to_save = []

                # write out the batch and reset
                print(' -> writing: ' + PREPROCESSED_NEW_DATA_FOLDER + 
                                        'preprocessed_new_TSA_scans-tz{}-{}-{}-b{}.npy'.format( 
                                        tz_num+1,
                                        len(threat_zone_examples[0][1][0]),
                                        len(threat_zone_examples[0][1][1]), 
                                        batch_num))

                # get this tz's examples
                tz_examples = [example for example in threat_zone_examples if example[0] == [tz_num]]

                # drop unused columns
                tz_examples_to_save.append([[features_label[1], features_label[2]] 
                                            for features_label in tz_examples])

                # save batch.  Note that the trainer looks for tz{} where {} is a 
                # tz_num 1 based in the minibatch file to select which batches to 
                # use for training a given threat zone
                np.save(PREPROCESSED_NEW_DATA_FOLDER +  'preprocessed_new_TSA_scans-tz{}-{}-{}-b{}.npy'.format(tz_num+1, len(threat_zone_examples[0][1][0]),len(threat_zone_examples[0][1][1]),batch_num), tz_examples_to_save)
                del tz_examples_to_save

            #reset for next batch 
            del threat_zone_examples
            threat_zone_examples = []
            batch_num += 1
    
    # we may run out of subjects before we finish a batch, so we write out 
    # the last batch stub
    if (len(threat_zone_examples) > 0):
        for tz_num, tz in enumerate(tsa.zone_slice_list):

            tz_examples_to_save = []

            # write out the batch and reset
            print(' -> writing: ' + PREPROCESSED_NEW_DATA_FOLDER 
                    + 'preprocessed_new_TSA_scans-tz{}-{}-{}-b{}.npy'.format(tz_num+1, 
                      len(threat_zone_examples[0][1][0]),
                      len(threat_zone_examples[0][1][1]),batch_num))

            # get this tz's examples
            tz_examples = [example for example in threat_zone_examples if example[0] == 
                           [tz_num]]

            # drop unused columns
            tz_examples_to_save.append([[features_label[1], features_label[2]] 
                                        for features_label in tz_examples])

            #save batch
            np.save(PREPROCESSED_NEW_DATA_FOLDER + 
                    'preprocessed_new_TSA_scans-tz{}-{}-{}-b{}.npy'.format(tz_num+1, 
                                                     len(threat_zone_examples[0][1][0]),
                                                     len(threat_zone_examples[0][1][1]), 
                                                     batch_num), 
                                                     tz_examples_to_save)
# unit test ---------------------------------------
#preprocess_tsa_data()

In [129]:
#train_conv_net()
#feature_batch
#image_test = tsa.read_data(INPUT_FOLDER + '/' + sub_list[0] + '.aps')

preprocess_tsa_new_data()


--------------------------------------------------------------
t+> 0.000 |Reading images for subject #: 0367394485447c1c3485359ba71f52cb
--------------------------------------------------------------
Threat Zone:Image -> 0:0
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=5.0015478336717933e-05
-> converting to grayscale
-> shape (660, 512)|mean=14.236283735795455
-> spreading spectrum
-> shape (660, 512)|mean=19.70759647253788
-> masking image
-> shape (660, 512)|mean=0.5041045217803031
-> cropping image
-> shape (250, 250)|mean=2.555152
-> normalizing image
-> shape (250, 250)|mean=0.010020203921568627
-> zero centering
-> shape (250, 250)|mean=-0.004306796078431377
-> appending example to threat zone 0
-> shape 1:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 0:1
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=4.920102219330147e-05
-> converting to grayscale
-> shape (660, 512)|mean=11.7

-> shape (660, 512)|mean=19.612976444128787
-> masking image
-> shape (660, 512)|mean=0.5258049242424242
-> cropping image
-> shape (250, 250)|mean=0.0
-> normalizing image
-> shape (250, 250)|mean=0.0
-> zero centering
-> shape (250, 250)|mean=-0.014327000000000003
-> appending example to threat zone 1
-> shape 19:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 1:11
Threat Zone Label -> [1 0]
-> No view of tz:1 in img:11. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 1:12
Threat Zone Label -> [1 0]
-> No view of tz:1 in img:12. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 1:13
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=5.0279253628104925e-05
-> converting to grayscale
-> shape (660, 512)|mean=13.619019294507575
-> spreading spectrum
-> shape (660, 512)|mean=18.47692353219697
-> masking image
-> shape (660, 512)|mean=0.742294034

------------------------------------------------
Threat Zone:Image -> 3:2
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=5.020459138904698e-05
-> converting to grayscale
-> shape (660, 512)|mean=11.899562026515152
-> spreading spectrum
-> shape (660, 512)|mean=17.78817175662879
-> masking image
-> shape (660, 512)|mean=0.4997573390151515
-> cropping image
-> shape (250, 250)|mean=0.0
-> normalizing image
-> shape (250, 250)|mean=0.0
-> zero centering
-> shape (250, 250)|mean=-0.014327000000000003
-> appending example to threat zone 3
-> shape 37:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 3:3
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=4.861661363975145e-05
-> converting to grayscale
-> shape (660, 512)|mean=11.683365885416666
-> spreading spectrum
-> shape (660, 512)|mean=16.691432883522726
-> masking image
-> shape (660, 512)|mean=0.23155184659090908
-> cropping image
-> shape (25

Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=5.0015478336717933e-05
-> converting to grayscale
-> shape (660, 512)|mean=14.236283735795455
-> spreading spectrum
-> shape (660, 512)|mean=19.70759647253788
-> masking image
-> shape (660, 512)|mean=1.4678385416666666
-> cropping image
-> shape (250, 250)|mean=7.643392
-> normalizing image
-> shape (250, 250)|mean=0.029974086274509804
-> zero centering
-> shape (250, 250)|mean=0.015647086274509794
-> appending example to threat zone 5
-> shape 55:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 5:1
Threat Zone Label -> [1 0]
-> No view of tz:5 in img:1. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 5:2
Threat Zone Label -> [1 0]
-> No view of tz:5 in img:2. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 5:3
Threat Zone Label -> [1 0]
-> No view of tz:5 in img:3. Skipping to next...
-----

-> shape (660, 512)|mean=11.782134824810607
-> spreading spectrum
-> shape (660, 512)|mean=18.099020478219696
-> masking image
-> shape (660, 512)|mean=0.49646957859848484
-> cropping image
-> shape (250, 250)|mean=2.489872
-> normalizing image
-> shape (250, 250)|mean=0.009764203921568628
-> zero centering
-> shape (250, 250)|mean=-0.004562796078431373
-> appending example to threat zone 7
-> shape 73:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 7:2
Threat Zone Label -> [1 0]
-> No view of tz:7 in img:2. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 7:3
Threat Zone Label -> [1 0]
-> No view of tz:7 in img:3. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 7:4
Threat Zone Label -> [1 0]
-> No view of tz:7 in img:4. Skipping to next...
------------------------------------------------
Threat Zone:Image -> 7:5
Threat Zone Label -> [1 0]
-> No view of tz:7 in img:5. 

...

-> shape (660, 512)|mean=18.47692353219697
-> masking image
-> shape (660, 512)|mean=2.104714133522727
-> cropping image
-> shape (250, 250)|mean=11.0148
-> normalizing image
-> shape (250, 250)|mean=0.04319529411764706
-> zero centering
-> shape (250, 250)|mean=0.028868294117647058
-> appending example to threat zone 10
-> shape 110:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 10:14
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=4.867836833000183e-05
-> converting to grayscale
-> shape (660, 512)|mean=11.694575639204546
-> spreading spectrum
-> shape (660, 512)|mean=16.43052793560606
-> masking image
-> shape (660, 512)|mean=1.4675189393939394
-> cropping image
-> shape (250, 250)|mean=7.569664
-> normalizing image
-> shape (250, 250)|mean=0.029684956862745093
-> zero centering
-> shape (250, 250)|mean=0.015357956862745093
-> appending example to threat zone 10
-> shape 111:3:1:250:250:2
----------------------------

-> zero centering
-> shape (250, 250)|mean=0.010448905882352942
-> appending example to threat zone 12
-> shape 127:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 12:1
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=4.920102219330147e-05
-> converting to grayscale
-> shape (660, 512)|mean=11.782134824810607
-> spreading spectrum
-> shape (660, 512)|mean=18.099020478219696
-> masking image
-> shape (660, 512)|mean=1.1240944602272727
-> cropping image
-> shape (250, 250)|mean=5.834336
-> normalizing image
-> shape (250, 250)|mean=0.022879749019607844
-> zero centering
-> shape (250, 250)|mean=0.008552749019607843
-> appending example to threat zone 12
-> shape 128:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 12:2
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=5.020459138904698e-05
-> converting to grayscale
-> shape (660, 512)|mean=11.899562026515152
->

-> appending example to threat zone 13
-> shape 145:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 13:7
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=6.016737097525038e-05
-> converting to grayscale
-> shape (660, 512)|mean=13.555110677083333
-> spreading spectrum
-> shape (660, 512)|mean=20.67049005681818
-> masking image
-> shape (660, 512)|mean=1.3048443418560607
-> cropping image
-> shape (250, 250)|mean=5.501616
-> normalizing image
-> shape (250, 250)|mean=0.021574964705882355
-> zero centering
-> shape (250, 250)|mean=0.007247964705882347
-> appending example to threat zone 13
-> shape 146:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 13:8
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=6.255197513382882e-05
-> converting to grayscale
-> shape (660, 512)|mean=18.02766335227273
-> spreading spectrum
-> shape (660, 512)|mean=23.803841145833335
->

-> converting to grayscale
-> shape (660, 512)|mean=14.236283735795455
-> spreading spectrum
-> shape (660, 512)|mean=19.70759647253788
-> masking image
-> shape (660, 512)|mean=0.48084457859848484
-> cropping image
-> shape (250, 250)|mean=2.421232
-> normalizing image
-> shape (250, 250)|mean=0.009495027450980392
-> zero centering
-> shape (250, 250)|mean=-0.004831972549019611
-> appending example to threat zone 15
-> shape 162:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 15:1
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=4.920102219330147e-05
-> converting to grayscale
-> shape (660, 512)|mean=11.782134824810607
-> spreading spectrum
-> shape (660, 512)|mean=18.099020478219696
-> masking image
-> shape (660, 512)|mean=0.4580314867424242
-> cropping image
-> shape (250, 250)|mean=2.297888
-> normalizing image
-> shape (250, 250)|mean=0.009011325490196077
-> zero centering
-> shape (250, 250)|mean=-0.00531567450980

-> cropping image
-> shape (250, 250)|mean=13.211568
-> normalizing image
-> shape (250, 250)|mean=0.051810070588235294
-> zero centering
-> shape (250, 250)|mean=0.03748307058823531
-> appending example to threat zone 16
-> shape 178:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 16:12
Threat Zone Label -> [1 0]
-> reorienting base image
-> shape (660, 512)|mean=4.7842349886195734e-05
-> converting to grayscale
-> shape (660, 512)|mean=14.218560606060606
-> spreading spectrum
-> shape (660, 512)|mean=17.703707978219697
-> masking image
-> shape (660, 512)|mean=2.442755681818182
-> cropping image
-> shape (250, 250)|mean=12.188448
-> normalizing image
-> shape (250, 250)|mean=0.047797835294117644
-> zero centering
-> shape (250, 250)|mean=0.03347083529411766
-> appending example to threat zone 16
-> shape 179:3:1:250:250:2
------------------------------------------------
Threat Zone:Image -> 16:13
Threat Zone Label -> [1 0]
-> reorienting base image

In [139]:
FILE_LIST = [f for f in os.listdir(PREPROCESSED_NEW_DATA_FOLDER)if re.search(re.compile('-tz' + str(THREAT_ZONE) + '-'), f)]
# train_test_split = len(FILE_LIST) -  max(int(len(FILE_LIST)*TRAIN_TEST_SPLIT_RATIO),1)
# TRAIN_SET_FILE_LIST = FILE_LIST[:train_test_split]
# TEST_SET_FILE_LIST = FILE_LIST[train_test_split:]
# TRAIN_SET_FILE_LIST = TEST_SET_FILE_LIST

In [106]:
TRAIN_SET_FILE_LIST[0]

'preprocessed_TSA_scans-tz1-250-250-b1.npy'

In [1]:
for i FILE_LIST:
    print i

SyntaxError: invalid syntax (<ipython-input-1-7d870919069d>, line 1)

In [141]:
#f_in = FILE_LIST[1]
f_in = FILE_LIST[0]
feature_batch, label_batch = input_pipeline(f_in, PREPROCESSED_NEW_DATA_FOLDER)

In [142]:
len(feature_batch)

11

In [143]:
feature_batch = feature_batch.reshape(-1, IMAGE_DIM, IMAGE_DIM, 1)

x_predi = model.predict(feature_batch)

In [144]:
x_predi

array([[ 0.56703085,  0.43296921],
       [ 0.59470272,  0.40529731],
       [ 0.62208891,  0.37791112],
       [ 0.51924247,  0.4807575 ],
       [ 0.51924247,  0.4807575 ],
       [ 0.51924247,  0.4807575 ],
       [ 0.51924247,  0.4807575 ],
       [ 0.51924247,  0.4807575 ],
       [ 0.70567364,  0.29432639],
       [ 0.60297835,  0.39702162],
       [ 0.56430954,  0.43569043]], dtype=float32)

In [78]:
sub_list[0:1]

['0367394485447c1c3485359ba71f52cb']

<H3>Wrap Up!</H3>

Alright, now its time to cut it loose.  I convert this notebook into a script and let the magic begin.  So far I have run training against the first three threat zones (1-3),  I am currently seeing validation accuracy in the 92-96% range. If you checked out the [EDA](http://s://www.kaggle.com/jbfarrar/exploratory-data-analysis-and-example-generation) you'll recall that the probabilities for the first three zones are 11.6%, 11%, 9.1% respectively.  So a model that just predicts "no contraband", should perform at 88.4%, 89%, and 90.8%.  So while it appears we may be getting some predictive value, much work would be needed to drive those accuracy numbers higher.  With the pipeline working, I'm going to fire up a meaningful training run andI will update this section with a fullsome view by threat zone of the accuracy, once I've run enough epochs to have useful view.

If you've found this helpful, I hope you'll give me an up vote!

Good Luck!

